# Introduction to Data Science
## Text classification
***

Read in some packages.

In [1]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

### Data
Let's read in some text data and take a look at what we have.

ESCAPE AND QUOTE CHARACTER EXPLANATIONS?

In [20]:
data = pd.read_csv("data/spam_ham.csv", quotechar="'", escapechar="\\")

The target is whether or not a record should be considered as spam. This is recorded as the string 'spam' or 'ham'. To make it a little easier, 

In [21]:
data['spam'] = pd.Series(data['spam'] == 'spam', dtype=int)